# OpenAI Responses API: Advanced Tweet Analysis with File & Web Search Integration

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Connecting to a MongoDB database** to store and retrieve tweets.
- **Extracting tweets** and converting them into a structured format for further analysis.
- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval 
and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API and interact with a MongoDB database, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API
- **`pymongo`**: A Python driver for MongoDB to store and retrieve tweets.

In [1]:
pip install openai pymongo -q

Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
# tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

## Extract Tweets from MongoDB

In [5]:
filter={

    
}
project={
    'tweet.text': 1,
    '_id':0
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

After retrieving tweets from MongoDB, we convert the query result into a list format for easier processing.
The data is then serialized into a JSON-formatted string, ensuring it can be properly stored and shared across different services.
Using `io.BytesIO`, we create an in-memory JSON file, eliminating the need for disk writes.
This approach is particularly useful for applications that require temporary file storage, such as uploading datasets
to OpenAI's file search API or cloud storage for further analysis.

In [6]:
result_list = list(result)

# Convert result list to JSON string
json_data = json.dumps(result_list, default=str, indent=4)

# Create an in-memory JSON file
json_bytes = io.BytesIO(json_data.encode("utf-8"))
json_bytes.name = "tweet.json" 

In [7]:
print('Number of tweets: ',len(result_list))

Number of tweets:  69


## Initialize OpenAI Client

In [8]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [9]:
vector_store = client.vector_stores.create(
    name="tweet_base"
)
vector_store_id = vector_store.id
# print(vector_store_id)

### Upload Tweets File

In [10]:
file = client.files.create(
            file=json_bytes,
            purpose="assistants",)

file_id = file.id
# print(file_id)

### Attach File to Vector Store

In [11]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

# print(attach_status.id)

### Query the Vector Store

In [12]:
query = "the latest development in genai"

In [13]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

}
    },
    {
        "tweet": {
            "text": "Paul Lewis, brother of 2021 CAA Player of the
 Relevant score: 0.4290835602262984
\n\n\u26be\ufe0f Series win at home over ODU \n\ud83e\udd4e Series sweep at JMU \n\ud83c\udfbe Ran w
 Relevant score: 0.4163912111797036
{
        "tweet": {
            "text": "RT @HNlEHupY4Nr6hRM: JMU\u78ef\u5b50\u306b\u3066\uff92\uff
 Relevant score: 0.36305452975913405
\n\n\u26be\ufe0f Series win at home over ODU \n\ud83e\udd4e Series sweep at JMU \n\ud83c\udfbe Ran w
 Relevant score: 0.28946463475931905
\u0628\u0624\u062f\u0649\u2390\n\u22b5A4TU\u22b4\nJmU"
        }
    },
    {
        "tweet": {
   
 Relevant score: 0.2802026464100436


## OpenAI Response API

### Simple Response

In [14]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [15]:
display(Markdown(simple_response.output_text))

As of my latest update, the developments in Generative AI (GenAI) continue to be dynamic and expansive. Some key areas of advancement include:

1. **Multimodal Models**:
   - Models that can process and integrate multiple types of data inputs, such as text, images, and audio, are gaining traction. OpenAI's GPT-4 and similar models exemplify this trend by handling tasks across different media.

2. **Increased Accessibility**:
   - Platforms are providing user-friendly interfaces for GenAI technologies, making it easier for developers and businesses to integrate AI into applications without deep technical expertise.

3. **Enhanced Personalization**:
   - GenAI is being used to create highly personalized content across industries, from marketing to entertainment, adapting outputs based on user preferences and behaviors.

4. **Ethical and Responsible AI**:
   - There is an increased focus on developing frameworks to ensure GenAI technologies are used responsibly. This includes bias mitigation, transparency, and regulatory compliance.

5. **Real-time Applications**:
   - Advancements in computational efficiency have enabled the use of GenAI in real-time applications, such as live translations, content creation, and interactive AI-driven environments.

6. **Creative Industries**:
   - Artists, writers, and musicians are leveraging GenAI to aid in the creative process, generating new works or augmenting their existing projects with AI-driven insights.

7. **Augmented Reality and Virtual Reality**:
   - GenAI is being integrated with AR and VR technologies to create immersive experiences and environments that respond dynamically to user interactions.

These developments are shaping the capabilities and applications of GenAI, influencing everything from business processes to everyday consumer interactions.

### File Search Response

In [16]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [17]:
display(Markdown(file_search_response.output_text))


I couldn't find specific information on the latest developments in generative AI from the provided file. If you have any other questions or need further assistance, feel free to ask!

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [18]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(web_search_response.output_text))

Generative AI has seen significant advancements across various domains, leading to innovative applications and enhanced capabilities. Here are some of the latest developments:

**Gaming and Real-Time AI Rendering**

Microsoft introduced WHAMM (World and Human Action MaskGIT Model), a generative AI designed for real-time gaming. Demonstrated through a Quake II demo, WHAMM generates visual outputs swiftly, supporting real-time user interaction. This model utilizes a MaskGIT-style framework to produce image tokens in parallel, reducing latency and computational demands. Although still in development, WHAMM signifies a notable step toward AI-assisted gaming experiences. ([tomshardware.com](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai))

**Business Applications and AI Integration**

At its Cloud Next conference, Google emphasized the practical business applications of its AI technology. Collaborations with over 500 customers, including Mattel, showcased how AI tools like BigQuery assist in analyzing product feedback and enhancing decision-making processes. Google also unveiled new offerings, such as the TPUv7 AI chip and A2A, a proposed protocol for inter-company AI agent communication, underscoring its commitment to providing tangible value in real-world business environments. ([axios.com](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai))

**Advancements in AI Hardware and Robotics**

Nvidia's GTC 2025 event highlighted several innovations, including the unveiling of Blackwell Ultra AI chips and the Groot N1 AI model for robotics. The company also introduced Blue, a robot powered by Newton, developed in collaboration with Google DeepMind and Disney. These developments emphasize Nvidia's focus on expanding AI capabilities and accelerating computing power, with projections of reaching $1 trillion in revenue by 2028. ([tomsguide.com](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai))

**3D Content Generation**

Tencent released a suite of open-source AI tools capable of converting text and images into 3D visuals. Leveraging its Hunyuan3D-2.0 technology, these models can generate high-quality 3D visuals in just 30 seconds. This initiative reflects China's growing competition in the generative AI space, offering high-performing models at reduced costs and catering to designers and game developers. ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai))

**Advancements in Language Models**

Google DeepMind unveiled Gemini, a multimodal large language model available in three sizes: Nano, Pro, and Ultra. Released in December 2023, Gemini integrates into Google's Bard chatbot and represents a significant step in challenging existing models like OpenAI's GPT-4. Subsequent updates, such as Gemini 2.0 Flash and Gemini 2.5, introduced expanded multimodality, including image and audio generation, and enhanced reasoning capabilities. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_DeepMind?utm_source=openai))


## Recent Developments in Generative AI:
- [You can now play a real-time AI-rendered Quake II in your browser - Microsoft's WHAMM offers generative AI for games](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai)
- [Google touts AI's real-world business model](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai)
- [Nvidia GTC 2025 - Blackwell Ultra, Groot N1, self-driving cars and more from Jensen Huang's keynote](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai) 

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [20]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Generative AI has seen significant advancements across various domains, leading to innovative applic

### Continue Query with Web Search

In [21]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [22]:
display(Markdown(continue_search_response.output_text))

Certainly, here are some recent developments in generative AI:

**Policy and Regulation**

- **IBM Advocates for AI Accountability**: IBM CEO Arvind Krishna has called for regulatory measures to hold AI companies accountable for the potential harms caused by their technologies. He emphasized that IBM itself would not be exempt from such regulations, highlighting the need for industry-wide responsibility. ([opensourcebiology.eu](https://opensourcebiology.eu/2023/10/13/the-latest-in-generative-ai-news/?utm_source=openai))

- **European Union's Three-Tiered AI Oversight**: The European Union is considering a structured approach to regulate generative AI models. This proposed framework includes:
  - A first tier encompassing all foundational models.
  - A second tier targeting "highly capable" systems, defined by the computational power used in training large language models.
  - A third tier focusing on popular AI models, determined by their total user base. ([adweek.com](https://www.adweek.com/programmatic/ai-watch-weekly-updates-for-the-latest-in-generative-ai-news-2/?utm_source=openai))

**Technological Developments**

- **BBC Restricts AI Web Crawlers**: The BBC has announced plans to block web crawlers from OpenAI and Common Crawl from accessing its websites. This decision reflects concerns over content usage and the implications of AI training on proprietary material. ([opensourcebiology.eu](https://opensourcebiology.eu/2023/10/13/the-latest-in-generative-ai-news/?utm_source=openai))

- **Adobe Enhances Firefly Suite**: Adobe has introduced new tools within its Firefly suite, which has generated over 3 billion images since March. These enhancements aim to assist marketers in expediting tasks such as creating mood boards and product mock-ups, showcasing the practical applications of generative AI in creative industries. ([opensourcebiology.eu](https://opensourcebiology.eu/2023/10/13/the-latest-in-generative-ai-news/?utm_source=openai))

**Business Applications**

- **Gymshark Integrates AI for Customer Guidance**: UK-based fitness apparel brand Gymshark is leveraging Google's AI platform, Vertex AI, to develop expert assistants that guide consumers through product selection. Additionally, the brand plans to enhance its training app by enabling users to log activities via text and receive personalized insights to achieve their fitness goals. ([opensourcebiology.eu](https://opensourcebiology.eu/2023/10/13/the-latest-in-generative-ai-news/?utm_source=openai))

- **Amazon Utilizes AI for Delivery Optimization**: Amazon is employing generative AI to optimize delivery routes and enhance warehouse robotics, aiming to improve same-day delivery services. This integration of AI technologies seeks to streamline operations and create more ergonomic working environments for employees. ([highpeaksw.com](https://highpeaksw.com/generative-ai-news-breaking-developments-in-ai-powered-innovation/?utm_source=openai))

**Industry Trends**

- **India's Leadership in AI Adoption**: Research by SAP reveals that small and medium-sized businesses (SMBs) in India are leading in global AI adoption. This trend underscores the country's proactive approach to integrating AI technologies across various sectors. ([highpeaksw.com](https://highpeaksw.com/generative-ai-news-breaking-developments-in-ai-powered-innovation/?utm_source=openai))

- **Generative AI's Impact on the 2024 Presidential Race**: The 2024 presidential race has seen the emergence of AI-generated imagery featuring candidates in outlandish scenarios. While often comical, these AI-generated memes sometimes carry serious political undertones, reflecting the complex role of generative AI in political discourse. ([highpeaksw.com](https://highpeaksw.com/generative-ai-news-breaking-developments-in-ai-powered-innovation/?utm_source=openai))


## Recent Developments in Generative AI:
- [In 2024, artificial intelligence was all about putting AI tools to work](https://apnews.com/article/0b6ab89193265c3f60f382bae9bbabc9?utm_source=openai) 

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [23]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [24]:
display(Markdown(combined_search_response.output_text))

Generative AI has experienced significant advancements across various domains, leading to innovative applications and enhanced capabilities. Here are some of the latest developments:

**1. Real-Time AI in Gaming:**
Microsoft introduced WHAMM (World and Human Action MaskGIT Model), a generative AI model designed for real-time gaming. Demonstrated through a Quake II demo, WHAMM generates visual outputs swiftly, supporting real-time user interaction. This model utilizes a MaskGIT-style framework to produce image tokens in parallel, reducing latency and computational demands. Although the demo exhibits some limitations like low frame rates and input lag, it marks a significant step toward AI-assisted gaming experiences. ([tomshardware.com](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai))

**2. AI Integration in Business Applications:**
At its Cloud Next conference, Google emphasized the practical business applications of its AI technology. Collaborating with over 500 customers, including Mattel, Google showcased how its BigQuery AI tool analyzed feedback on products like the Barbie Dreamhouse. Additionally, Google unveiled new offerings such as the TPUv7 AI chip and A2A, a proposed protocol for inter-company AI agent communication, underscoring its commitment to providing tangible value in real-world business environments. ([axios.com](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai))

**3. AI Hardware and Robotics Innovations:**
Nvidia's GTC 2025 event highlighted several innovations, including the unveiling of Blackwell Ultra AI chips and the Groot N1 AI model for robotics. The company also introduced Blue, a robot developed in collaboration with Google DeepMind and Disney. These advancements emphasize Nvidia's focus on expanding AI capabilities and accelerating computing power, with projections of reaching $1 trillion in revenue by 2028. ([tomsguide.com](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai))

**4. Open-Source 3D Generation Tools:**
Tencent released a suite of open-source AI tools capable of converting text and images into 3D visuals. Leveraging its Hunyuan3D-2.0 technology, these models can generate high-quality 3D visuals in just 30 seconds. This initiative signifies China's advancements in generative AI, offering high-performing models at reduced costs and catering to designers and game developers. ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai))

**5. Advancements in AI Models:**
In December 2023, Google unveiled Gemini, a multimodal AI model available in versions like Ultra, Pro, Flash, and Nano. Integrated into its Bard chatbot, Gemini represents a significant step in AI model development. Similarly, Anthropic released the Claude 3 family of large language models in March 2024, demonstrating substantial improvements across various benchmarks and outperforming leading models from OpenAI and Google. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Generative_artificial_intelligence?utm_source=openai))

These developments reflect the rapid evolution of generative AI, with applications spanning gaming, business analytics, hardware innovations, and content creation, indicating a transformative impact across multiple industries.


## Recent Breakthroughs in Generative AI:
- [You can now play a real-time AI-rendered Quake II in your browser - Microsoft's WHAMM offers generative AI for games](https://www.tomshardware.com/video-games/you-can-now-play-a-real-time-ai-rendered-quake-ii-in-your-browser-microsofts-whamm-offers-generative-ai-for-games?utm_source=openai)
- [Google touts AI's real-world business model](https://www.axios.com/2025/04/09/google-ai-mattel-barbie?utm_source=openai)
- [Nvidia GTC 2025 - Blackwell Ultra, Groot N1, self-driving cars and more from Jensen Huang's keynote](https://www.tomsguide.com/computing/live/nvidia-gtc-2025-live?utm_source=openai) 